In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [ ]:
class EnhancedBabySimulator:
    def __init__(self, baby_ids, start_date, total_weeks=8):
        self.baby_ids = baby_ids
        self.start_date = start_date
        self.total_weeks = total_weeks
        self.developmental_stages = self._get_developmental_stages()
        self.env_profiles = self._initialize_environment_profiles()
        
    def _get_developmental_stages(self):
        """검색 결과[1] 기반 발달 단계별 수면 패턴"""
        return {
            # 주차: (기본수면확률, 낮잠횟수, 야간수면강화계수)
            0: (0.85, 4, 0.7),    # 신생아(0-1주)
            1: (0.83, 4, 0.75),   # 1주차 
            2: (0.80, 3, 0.8),    # 2주차
            4: (0.78, 3, 0.85),   # 4주차(1개월)
            8: (0.75, 3, 0.9),    # 2개월
            12: (0.70, 2, 0.95),  # 3개월
            16: (0.68, 2, 0.96),  # 4개월
            20: (0.65, 2, 0.97),  # 5개월
            24: (0.63, 1, 0.98),  # 6개월
            36: (0.60, 1, 0.99),  # 9개월
            48: (0.58, 1, 1.0)    # 12개월(1살)
        }
    
    def _get_stage_params(self, week):
        """주차에 맞는 발달 단계 매개변수 반환"""
        stages = sorted(self.developmental_stages.keys(), reverse=True)
        for s in stages:
            if week >= s:
                return self.developmental_stages[s]
        return (0.85, 4, 0.7)  # 기본값

    def _initialize_environment_profiles(self):
        """검색 결과[5] 기반 초기 환경 프로파일"""
        profiles = {}
        for bid in self.baby_ids:
            profiles[bid] = {
                'temp': np.random.uniform(26.0, 30.0),  # 초기 고온 설정
                'humidity': np.random.uniform(65.0, 75.0),
                'brightness': np.random.uniform(80.0, 100.0),
                'noise': np.random.randint(65, 85),
                'optimal_temp': np.random.normal(22.5, 1.0),  # 개인차 반영
                'adapt_speed': np.random.uniform(0.1, 0.3)   # 환경 적응 속도
            }
        return profiles
    
    def _adjust_environment(self, baby_id, is_sleeping, current_time, week):
        """발달 단계 반영 환경 조정"""
        profile = self.env_profiles[baby_id]
        hour = current_time.hour
        
        # 주차별 수면 패턴 적용
        base_sleep, nap_count, night_factor = self._get_stage_params(week)
        
        # 야간 수면 강화(검색 결과[4] 참조)
        night_boost = 1.2 if 22 <= hour < 6 else 0.8
        
        # 환경 조정 알고리즘
        if is_sleeping:
            # 수면 시 최적화 목표
            temp_target = profile['optimal_temp'] * night_boost
            noise_target = 45 + (week * 2)  # 주차가 증가할수록 소음 감소
        else:
            # 각성 시 활동성 환경
            temp_target = 24.0 + (week * 0.2)  # 주차 증가시 온도 상승
            noise_target = 55 - (week * 1)
        
        # 환경 변화량 계산(검색 결과[3]의 민감도 반영)
        delta_temp = (temp_target - profile['temp']) * profile['adapt_speed']
        delta_noise = (noise_target - profile['noise']) * 0.1
        
        # 실제 적용
        profile['temp'] += delta_temp * np.random.uniform(0.5, 1.5)
        profile['noise'] += int(delta_noise * np.random.uniform(0.8, 1.2))
        
        # 물리적 제한 조건
        profile['temp'] = np.clip(profile['temp'], 18.0, 32.0)
        profile['noise'] = np.clip(profile['noise'], 30, 90)

    def generate(self):
        data = []
        log_id = 0
        
        for baby_id in self.baby_ids:
            current_time = self.start_date
            end_time = self.start_date + timedelta(weeks=self.total_weeks)
            
            week = 0
            last_week_check = self.start_date
            is_sleeping = True
            consecutive_awake = 0
            
            while current_time < end_time:
                if (current_time - last_week_check).days >= 7:
                    week += 1
                    last_week_check = current_time
                
                # 환경 조정 메서드 호출
                self._adjust_environment(baby_id, is_sleeping, current_time, week)
                
                profile = self.env_profiles[baby_id]
                
                log_id += 1
                data.append({
                    "log_id": log_id,
                    "baby_id": baby_id,
                    "is_sleeping": is_sleeping,
                    "created_at": current_time,
                    "temperature": round(profile['temp'], 1),
                    "humidity": round(profile['humidity'], 1),
                    "brightness": round(profile['brightness'], 1),
                    "white_noise_level": profile['noise'],
                    "week": week
                })
                
                current_time += timedelta(seconds=10)
        
        return pd.DataFrame(data)

In [33]:
simulator = EnhancedBabySimulator(baby_ids=[1, 2, 3, 4, 5], start_date=datetime(2024,5,27), total_weeks=48)
df = simulator.generate()

In [34]:
df.to_csv('dummy_baby_sleep_data.csv', index=False, encoding='utf-8-sig')

In [35]:
df.head()

,log_id,baby_id,is_sleeping,created_at,temperature,humidity,brightness,white_noise_level,week
0,1,1,True,2024-05-27 00:00:00,26.5,72.6,94.2,68,0
1,2,1,True,2024-05-27 00:00:10,24.6,72.6,94.2,67,0
2,3,1,True,2024-05-27 00:00:20,23.3,72.6,94.2,65,0
3,4,1,True,2024-05-27 00:00:30,22.6,72.6,94.2,63,0
4,5,1,True,2024-05-27 00:00:40,21.6,72.6,94.2,62,0


In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14515200 entries, 0 to 14515199
Data columns (total 9 columns):
 #   Column             Dtype         
---  ------             -----         
 0   log_id             int64         
 1   baby_id            int64         
 2   is_sleeping        bool          
 3   created_at         datetime64[ns]
 4   temperature        float64       
 5   humidity           float64       
 6   brightness         float64       
 7   white_noise_level  int64         
 8   week               int64         
dtypes: bool(1), datetime64[ns](1), float64(3), int64(4)
memory usage: 899.8 MB


In [37]:
df.describe()

,log_id,baby_id,created_at,temperature,humidity,brightness,white_noise_level,week
count,1.451520e+07,1.451520e+07,14515200,1.451520e+07,1.451520e+07,1.451520e+07,1.451520e+07,1.451520e+07
mean,7.257600e+06,3.000000e+00,2024-11-10 23:59:55.000056832,1.852002e+01,6.930000e+01,9.242000e+01,7.631250e+01,2.350000e+01
min,1.000000e+00,1.000000e+00,2024-05-27 00:00:00,1.800000e+01,6.530000e+01,8.110000e+01,5.300000e+01,0.000000e+00
25%,3.628801e+06,2.000000e+00,2024-08-18 23:59:57.500000,1.800000e+01,6.650000e+01,9.250000e+01,6.000000e+01,1.175000e+01
50%,7.257600e+06,3.000000e+00,2024-11-10 23:59:55,1.800000e+01,6.920000e+01,9.420000e+01,8.300000e+01,2.350000e+01
75%,1.088640e+07,4.000000e+00,2025-02-02 23:59:52.500000,1.900000e+01,7.260000e+01,9.580000e+01,9.000000e+01,3.525000e+01
max,1.451520e+07,5.000000e+00,2025-04-27 23:59:50,2.700000e+01,7.290000e+01,9.850000e+01,9.000000e+01,4.700000e+01
std,4.190177e+06,1.414214e+00,NaN,6.645891e-01,3.088689e+00,5.995132e+00,1.520571e+01,1.385340e+01


In [38]:
df['week'].nunique()

48